### Basic Setup

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
df = pd.read_csv('/Users/eduardo/Desktop/AssetPricing/data.csv')
print(df.head())
print(df.describe())
print(df.columns)

                                  KEY FREQ REF_AREA CURRENCY PROVIDER_FM  \
0  YC.B.U2.EUR.4F.G_N_A.SV_C_YM.BETA0    B       U2      EUR          4F   
1  YC.B.U2.EUR.4F.G_N_A.SV_C_YM.BETA0    B       U2      EUR          4F   
2  YC.B.U2.EUR.4F.G_N_A.SV_C_YM.BETA0    B       U2      EUR          4F   
3  YC.B.U2.EUR.4F.G_N_A.SV_C_YM.BETA0    B       U2      EUR          4F   
4  YC.B.U2.EUR.4F.G_N_A.SV_C_YM.BETA0    B       U2      EUR          4F   

  INSTRUMENT_FM PROVIDER_FM_ID DATA_TYPE_FM TIME_PERIOD  OBS_VALUE  ...  \
0         G_N_A        SV_C_YM        BETA0  2004-09-06   5.410510  ...   
1         G_N_A        SV_C_YM        BETA0  2004-09-07   5.391886  ...   
2         G_N_A        SV_C_YM        BETA0  2004-09-08   5.385978  ...   
3         G_N_A        SV_C_YM        BETA0  2004-09-09   5.377333  ...   
4         G_N_A        SV_C_YM        BETA0  2004-09-10   5.355732  ...   

  UNIT_INDEX_BASE                                        COMPILATION  \
0             NaN  T

### Convert `df` from long to wide and keep useful columns

In [4]:
# Parse dates
df["TIME_PERIOD"] = pd.to_datetime(df["TIME_PERIOD"])

# Keep only euro area government bond Svensson params (this matches your sample)
mask = (
    (df["REF_AREA"] == "U2") &          # euro area
    (df["CURRENCY"] == "EUR") &
    (df["PROVIDER_FM"] == "4F") &
    (df["INSTRUMENT_FM"] == "G_N_A") &
    (df["PROVIDER_FM_ID"] == "SV_C_YM")
)

df_yc = df.loc[mask, ["TIME_PERIOD", "DATA_TYPE_FM", "OBS_VALUE"]].copy()

# Pivot: rows = date, columns = parameter (BETA0, BETA1, BETA2, BETA3, TAU1, TAU2, etc.)
yc_params = df_yc.pivot_table(
    index="TIME_PERIOD",
    columns="DATA_TYPE_FM",
    values="OBS_VALUE"
).sort_index()

# Optional: keep only the main Svensson parameters if you want
needed_cols = ["BETA0", "BETA1", "BETA2", "BETA3", "TAU1", "TAU2"]
yc_params = yc_params[needed_cols]

yc_params.head()

DATA_TYPE_FM,BETA0,BETA1,BETA2,BETA3,TAU1,TAU2
TIME_PERIOD,,,,,,
2004-09-06,5.410510,-3.462358,-0.361335,-0.466368,3.128331,1.489535
2004-09-07,5.391886,-3.450353,-0.372908,-0.271295,3.157868,1.548566
2004-09-08,5.385978,-3.447950,-0.346505,-0.198077,3.137857,1.553148
2004-09-09,5.377333,-3.432592,-0.382208,-0.293482,3.271392,1.521143
2004-09-10,5.355732,-3.395283,-0.375382,-0.507894,3.279876,1.479197
